#Classifying contextualized embeddings of the verbs

In [1]:
import json
import pandas as pd

Get the data from the TPAS corpus

In [2]:
def AccogliereCONT(filename):
    f = open(filename)
    r = json.load(f)
    data = r['data'] 
    sent_lab= []
    sent_lab_tok = []
    for x in data: 
        if x[0] == "accogliere":
            if x[1] == "1":
                tup = (0, x[2])
            if x[1] == "2":
                tup = (1, x[2])
            sent_lab.append(tup)
    for x in sent_lab:
        sen = x[1]
        sent_tokenized = sen.split(" ")
        for tok in sent_tokenized:
            if "accog" in tok or "accol" in tok or "ACCOL" in tok or "ACCOG" in tok:
                trip = (x[0], tok, x[1])
                sent_lab_tok.append(trip)
                break
    sent_lab_tok = list(set(sent_lab_tok))
    sentences = []
    labels = []
    tokens = []
    for x in sent_lab_tok:
        labels.append(x[0])
        sentences.append(x[1])
        tokens.append(x[2])    
    D = {'labels': labels, 'tokens': tokens, 'sentences': sentences}
    #df = pd.DataFrame(D)
    return sent_lab_tok, D

In [2]:
def Ritirare(filename):
    f = open(filename)
    r = json.load(f)
    data = r['data'] 
    sent_lab= []
    sent_lab_tok = []
    for x in data: 
        if x[0] == "ritirare":
            if x[1] == "1":
                tup = (0, x[2])
            if x[1] == "3":
                tup = (1, x[2])
            if x[1] == "5":
              tup = (2, x[2])
            sent_lab.append(tup)
    for x in sent_lab:
        sen = x[1]
        sent_tokenized = sen.split(" ")
        for tok in sent_tokenized:
            if "ritir" in tok:
                trip = (x[0], tok, x[1])
                sent_lab_tok.append(trip)
                break
    sent_lab_tok = list(set(sent_lab_tok))
    sentences = []
    labels = []
    tokens = []
    for x in sent_lab_tok:
        labels.append(x[0])
        sentences.append(x[1])
        tokens.append(x[2])    
    D = {'labels': labels, 'tokens': tokens, 'sentences': sentences}
    #df = pd.DataFrame(D)
    return sent_lab_tok, D

In [3]:
def ACCUSARE(filename):
    f = open(filename)
    r = json.load(f)
    data = r['data'] 
    sent_lab= []
    sent_lab_tok = []
    for x in data: 
        if x[0] == "accusare":
            if x[1] == "1":
                tup = (0, x[2])
            if x[1] == "2":
                tup = (1, x[2])
            if x[1] == "5":
              tup = (2, x[2])
            if x[1] == "6":
              tup = (3, x[2])
            sent_lab.append(tup)
    for x in sent_lab:
        sen = x[1]
        sent_tokenized = sen.split(" ")
        for tok in sent_tokenized:
            if "accus" in tok:
                trip = (x[0], tok, x[1])
                sent_lab_tok.append(trip)
                break
    sent_lab_tok = list(set(sent_lab_tok))
    sentences = []
    labels = []
    tokens = []
    for x in sent_lab_tok:
        labels.append(x[0])
        sentences.append(x[1])
        tokens.append(x[2])    
    D = {'labels': labels, 'tokens': tokens, 'sentences': sentences}
    #df = pd.DataFrame(D)
    return sent_lab_tok, D

Get contextualized embeddings of the verb (as in https://discuss.huggingface.co/t/generate-raw-word-embeddings-using-transformer-models-like-bert-for-downstream-process/2958) 

In [4]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
 import numpy as np
 import torch
 from transformers import AutoTokenizer, AutoModel
 
 def get_word_idx(sent: str, word: str):
     return sent.split(" ").index(word)

 def get_hidden_states(encoded, token_ids_word, model, layers):
     """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them."""
     with torch.no_grad():
         output = model(**encoded)
 
     # Get all hidden states
     states = output.hidden_states
     # Stack and sum all requested layers
     output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
     # Only select the tokens that constitute the requested word
     word_tokens_output = output[token_ids_word]
 
     return word_tokens_output.mean(dim=0)
 
 def get_word_vector(sent, idx, tokenizer, model, layers):
     """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
     encoded = tokenizer.encode_plus(sent, return_tensors="pt")
     # get all token idxs that belong to the word of interest
     token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
 
     return get_hidden_states(encoded, token_ids_word, model, layers)
 
 def main(sent, tok, layers=None):
     # Use last four layers by default
     layers = [-4, -3, -2, -1] if layers is None else layers

     tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-cased")
     model = AutoModel.from_pretrained("dbmdz/bert-base-italian-xxl-cased", output_hidden_states=True) 
     idx = get_word_idx(sent, tok)

     word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
     return word_embedding 


In [ ]:
if __name__ == '__main__':

  Ls, D = ACCUSARE("G:\My Drive\TESI codici\TPAS corpus.json")
  vectors = []

  for x in Ls:
    vec = main(x[2], x[1])
    vectors.append(vec)
  #prova_RITIRARE = main('ritira','Luca ritira i suoi giocattoli nella borsa verde').

In [9]:
  for x in range(768):
      A = []
      for S in vectors:
        A.append(S[x])
      D[str(x+1)] = A
      A=[]

  df = pd.DataFrame(D)
  print(df)

     labels                                             tokens    sentences  \
0         3  Al termine dell' intervento un Vigile del Fuoc...     accusato   
1         3  Molti bambini sottoposti a questo stress , se ...     accusare   
2         3  = = = = = = = = = = = = = = = = = = = = = = = ...     accusare   
3         3  Drammatico l' arrivo degli extracomunitari sul...     accusato   
4         3  Di successo in successo , che gli diedero fama...     accusare   
..      ...                                                ...          ...   
378       2  Porremo quindi come obiettivo della risorsa un...     accusare   
379       3  " Durante la festa ha accusato un malessere , ...     accusato   
380       1  Se si vorrà assegnare una punizione particolar...     accusare   
381       3  Di conseguenza accuseranno i sintomi tipici de...  accuseranno   
382       1  È accusato di lesioni gravissime e detenzione ...     accusato   

                   1                2              

Training the model using SVM

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df.drop(columns = ['sentences','labels', 'tokens'])
y = df.labels

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
len(X)

383

In [14]:
from sklearn.svm import SVC
from sklearn.metrics import recall_score
model = SVC()
model.fit(X_train, y_train)

SVC()

In [15]:
model.recall_score(X_test, y_test)

AttributeError: 'SVC' object has no attribute 'recall_score'

The classifier have an accuracy around 85%

In [16]:
model.predict(X_test)

array([3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 3, 3, 1, 0, 3, 3, 3, 3, 3, 2, 3, 3,
       0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 2, 3,
       3, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 1, 3, 3, 3,
       0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3])

In [ ]:
#now test the model on new instances! 

sent_ACCOGLIERE = ['Quanto al danno morale , la domanda dovrebbe invece essere accolta solo sulla basa dell accertamento , in astratto , della responsabilità del sig. R per il reato di cui all art. 388 CP',
             'Rilevato che il T.A.R./Sicilia - sezione I di Catania , con ordinanza n. 12/2000 del 15 gennaio 2000 , ha accolto la sospensiva di detto decreto su ricorso giurisdizionale proposto da quattro consiglieri e che con successive ordinanze nn ',
             'Fino ad ora abbiamo accolto ben 4 coppie di fratellini e sorelline e altri 5 bimbi , in età compresa tra 1 anno e 10 anni .',
             "La famiglia ha accolto il bambino come se fosse loro figlio, con amore e affetto"]

#prova l'ho definito sopra, è una frase che ha label 2

model.predict(prova_RITIRARE)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)


ValueError: ignored

#Classifying sentence embeddings


Here I use the sentence_transformers library

In [ ]:
pip install -U sentence-transformers

In [ ]:
import torch

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def ritornaembedding(sentence):
  embeddings = model.encode(sentence)
  return(embeddings)

# sentence può essere o una frase o una lista di frasi
# a seconda ritorna una matrice o un vettore

In [ ]:
# esempio

sentences = ['Quanto al danno morale , la domanda dovrebbe invece essere accolta solo sulla basa dell accertamento , in astratto , della responsabilità del sig. R per il reato di cui all art. 388 CP',
             'Rilevato che il T.A.R./Sicilia - sezione I di Catania , con ordinanza n. 12/2000 del 15 gennaio 2000 , ha accolto la sospensiva di detto decreto su ricorso giurisdizionale proposto da quattro consiglieri e che con successive ordinanze nn ',
             'Fino ad ora abbiamo accolto ben 4 coppie di fratellini e sorelline e altri 5 bimbi , in età compresa tra 1 anno e 10 anni .',
             "La famiglia ha accolto il bambino come se fosse loro figlio, con amore e affetto"]

prova = ritornaembedding(sentences)
len(prova)

#Create a Pandas DataFrame for my dataset

In [ ]:
import json
import pandas as pd

La tabella dei sentence embeddings è

```
sentence | vector | label_ind | label_name
```




In [ ]:
def Accogliere(filename):
    f = open(filename)
    r = json.load(f)
    data = r['data'] 
    labels = []
    label_names = ['']
    sentences = []

    for x in data: 
        if x[0] == "accogliere":
            if x[1] == "1":
                labels.append(0)
                sentences.append(x[2])
            if x[1] == "2":
                labels.append(1) 
                sentences.append(x[2])
    embeddings = model.encode(sentences)
    print(type(embeddings))
    D = {"sentences": sentences, "labels": labels}
    for x in range(384):
      A = []
      for S in embeddings:
        A.append(S[x])
      D[str(x+1)] = A
      A=[]

    return D 

In [ ]:
Ds = Accogliere('/content/TPAS corpus.json')
df = pd.DataFrame(Ds)
print(df)

#Training the model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['sentences','labels'], axis='columns')
y = df.labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
len(X_test)

In [ ]:
from sklearn.svm import SVC
model = SVC()

In [ ]:
model.fit(X_train.values, y_train.values)

SVC()

In [ ]:
model.score(X_test.values, y_test.values)

0.6190476190476191

In [ ]:
model.predict(prova)

array([1, 1, 0, 0])